# ClimateWins Weather Prediction : Keras Layered Model (RNN/LSTM)

### The following Script Contains:
#### 1. Import Libraries and Data¶
#### 2. Data Wrangling
#### 3. Data Splitting
#### 4. Keras Model Creation
#### 5. Compiling and Running Model
#### 6. Confusion Matrix Creation
#### 8. Keras Model Retrials (Until Convergence)

### 1. Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Set display options to show all columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Create path for faster data importation
# Create a path for faster data importation
path = r'C:\Users\kenzy\OneDrive\Desktop\Data Analytics Course 2.0\Basics of Machine Learning for Analysts\10-2024 ClimateWins'
X = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'weather_cleaned.csv'))
prediction = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
# confirm import
X.head()

BASEL_cloud_cover  BASEL_humidity  BASEL_pressure  BASEL_global_radiation  \
0                  7            0.85           1.018                    0.32   
1                  6            0.84           1.018                    0.36   
2                  8            0.90           1.018                    0.18   
3                  3            0.92           1.018                    0.58   
4                  6            0.95           1.018                    0.65   

   BASEL_precipitation  BASEL_sunshine  BASEL_temp_mean  BASEL_temp_min  \
0                 0.09             0.7              6.5             0.8   
1                 1.05             1.1              6.1             3.3   
2                 0.30             0.0              8.5             5.1   
3                 0.00             4.1              6.3             3.8   
4                 0.14             5.4              3.0            -0.7   

   BASEL_temp_max  BELGRADE_cloud_cover  BELGRADE_humidity  BELGRADE_pressure  \
0            10.9                     1               0.81             1.0195   
1            10.1                     6               0.84             1.0172   
2             9.9                     6               0.77             1.0179   
3            10.6                     8               0.93             1.0268   
4             6.0                     8               0.99             1.0286   

   BELGRADE_global_radiation  BELGRADE_precipitation  BELGRADE_sunshine  \
0                       0.88                    0.00                7.0   
1                       0.25                    0.00                0.0   
2                       0.67                    0.00                3.5   
3                       0.25                    0.00                0.0   
4                       0.25                    0.06                0.0   

   BELGRADE_temp_mean  BELGRADE_temp_min  BELGRADE_temp_max  \
0                 3.7               -0.9                7.9   
1                 2.9                2.2                4.4   
2                 3.1               -0.5                6.4   
3                 2.0               -2.0                3.0   
4                 2.0                0.7                2.8   

   BUDAPEST_cloud_cover  BUDAPEST_humidity  BUDAPEST_pressure  \
0                     4               0.67              1.017   
1                     4               0.67              1.017   
2                     4               0.67              1.017   
3                     4               0.67              1.017   
4                     4               0.67              1.017   

   BUDAPEST_global_radiation  BUDAPEST_precipitation  BUDAPEST_sunshine  \
0                       0.44                    0.01                2.3   
1                       0.18                    0.31                0.0   
2                       0.30                    0.00                0.6   
3                       0.19                    0.00                0.0   
4                       0.19                    0.00                0.0   

   BUDAPEST_temp_mean  BUDAPEST_temp_min  BUDAPEST_temp_max  \
0                 2.4               -0.4                5.1   
1                 2.3                1.4                3.1   
2                 2.7                1.7                5.3   
3                 2.0                0.4                4.4   
4                 2.5                1.1                5.3   

   DEBILT_cloud_cover  DEBILT_humidity  DEBILT_pressure  \
0                   7             0.85           1.0032   
1                   8             0.90           1.0056   
2                   6             0.92           1.0165   
3                   8             0.95           1.0265   
4                   6             0.90           1.0243   

   DEBILT_global_radiation  DEBILT_precipitation  DEBILT_sunshine  \
0                     0.07                  0.25              0.0   
1                     0.14              

In [4]:
# check shape
X.shape

(22950, 135)

In [5]:
# check second dataset
prediction.head()

DATE  BASEL_pleasant_weather  BELGRADE_pleasant_weather  \
0  19600101                       0                          0   
1  19600102                       0                          0   
2  19600103                       0                          0   
3  19600104                       0                          0   
4  19600105                       0                          0   

   BUDAPEST_pleasant_weather  DEBILT_pleasant_weather  \
0                          0                        0   
1                          0                        0   
2                          0                        0   
3                          0                        0   
4                          0                        0   

   DUSSELDORF_pleasant_weather  HEATHROW_pleasant_weather  \
0                            0                          0   
1                            0                          0   
2                            0                          0   
3                            0                          0   
4                            0                          0   

   KASSEL_pleasant_weather  LJUBLJANA_pleasant_weather  \
0                        0                           0   
1                        0                           0   
2                        0                           0   
3                        0                           0   
4                        0                           0   

   MAASTRICHT_pleasant_weather  MADRID_pleasant_weather  \
0                            0                        0   
1                            0                        0   
2                            0                        0   
3                            0                        0   
4                            0                        0   

   MUNCHENB_pleasant_weather  OSLO_pleasant_weather  \
0                          0                      0   
1                          0                      0   
2                          0                      0   
3                          0                      0   
4                          0                      0   

   SONNBLICK_pleasant_weather  STOCKHOLM_pleasant_weather  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           0   

   VALENTIA_pleasant_weather  
0                          0  
1                          0  
2                          0  
3                          0  
4                          0

In [6]:
prediction.shape

(22950, 16)

### 02. Data Wrangling and Reshaping

### 3. Data Wrangling

In [7]:
# Drop DATE column from answers

prediction.drop(columns = 'DATE', inplace = True)

In [8]:
prediction.shape

(22950, 15)

In [9]:
y = prediction

In [10]:
X.shape

(22950, 135)

In [11]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [12]:
X = X.reshape(-1,15,9)

In [13]:
# Verify Shape
X.shape

(22950, 15, 9)

In [14]:
# Verify Shape
y.shape

(22950, 15)

In [15]:
X

array([[[  7.    ,   0.85  ,   1.018 , ...,   6.5   ,   0.8   ,
          10.9   ],
        [  1.    ,   0.81  ,   1.0195, ...,   3.7   ,  -0.9   ,
           7.9   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.4   ,  -0.4   ,
           5.1   ],
        ...,
        [  4.    ,   0.73  ,   1.0304, ...,  -5.9   ,  -8.5   ,
          -3.2   ],
        [  5.    ,   0.98  ,   1.0114, ...,   4.2   ,   2.2   ,
           4.9   ],
        [  5.    ,   0.88  ,   1.0003, ...,   8.5   ,   6.    ,
          10.9   ]],

       [[  6.    ,   0.84  ,   1.018 , ...,   6.1   ,   3.3   ,
          10.1   ],
        [  6.    ,   0.84  ,   1.0172, ...,   2.9   ,   2.2   ,
           4.4   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.3   ,   1.4   ,
           3.1   ],
        ...,
        [  6.    ,   0.97  ,   1.0292, ...,  -9.5   , -10.5   ,
          -8.5   ],
        [  5.    ,   0.62  ,   1.0114, ...,   4.    ,   3.    ,
           5.    ],
        [  7.    ,   0.91  ,   1.0007, ...,   8.

### 3. Data Splitting

In [16]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


### 4. Keras Model Creation

In [18]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh')) # Don't use relu here!

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 14, 64)              │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 7, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 15)                  │             975 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,215 (137.56 KB)

 Trainable params: 35,215 (137.56 KB)

 Non-trainable params: 0 (0.00 B)

### 5. Compiling and Running Model

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 7s - 6ms/step - accuracy: 0.1283 - loss: 24.1691
Epoch 2/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0949 - loss: 24.8244
Epoch 3/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0299 - loss: 24.8137
Epoch 4/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0260 - loss: 24.1067
Epoch 5/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0544 - loss: 25.2862
Epoch 6/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0806 - loss: 24.6753
Epoch 7/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1018 - loss: 24.9529
Epoch 8/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0564 - loss: 24.8119
Epoch 9/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1190 - loss: 24.9815
Epoch 10/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1425 - loss: 24.7889
Epoch 11/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1217 - loss: 24.7857
Epoch 12/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1755 - loss: 24.6927
Epoch 13/30
1076/1076 - 5s - 4ms/step - accuracy: 0.2400 - loss: 24.8767
Epoch 14/30
1076/1076 - 5s - 4ms/step - accuracy: 0.2955 - l

### 6. Confusion Matrix Creation

In [22]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [23]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [24]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BUDAPEST  VALENTIA
True                          
BASEL           3681         1
BELGRADE        1092         0
BUDAPEST         213         1
DEBILT            82         0
DUSSELDORF        29         0
HEATHROW          82         0
KASSEL            11         0
LJUBLJANA         61         0
MAASTRICHT         9         0
MADRID           458         0
MUNCHENB           8         0
OSLO               5         0
STOCKHOLM          4         0
VALENTIA           1         0


In [25]:
epochs = 30
batch_size = 16
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)                    │ (None, 14, 4)               │              76 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 14, 16)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 7, 16)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15)                  │           1,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0905 - loss: 697.6992
Epoch 2/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1176 - loss: 6546.9336
Epoch 3/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1316 - loss: 21063.2676
Epoch 4/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1283 - loss: 44454.2578
Epoch 5/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1264 - loss: 80002.2109
Epoch 6/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1311 - loss: 122232.7969
Epoch 7/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1305 - loss: 178294.6094
Epoch 8/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1297 - loss: 246433.4062
Epoch 9/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1354 - loss: 328033.3125
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1310 - loss: 426522.9062
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1328 - loss: 541408.0625
Epoch 12/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1329 - loss: 674206.3125
Epoch 13/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1330 - loss: 820982.8125
Epoch 14/30
1076

In [29]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [30]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         506       637        50     824           5        21     758   
BELGRADE       32       384        29     182           0         0     258   
BUDAPEST       11        68         9      62           0         0      30   
DEBILT          2         5         7      37           0         0      15   
DUSSELDORF      2         2         1      14           0         0       4   
HEATHROW       21         4         1      42           0         0       8   
KASSEL          0         3         1       6           0         0       0   
LJUBLJANA      16        26         0      10           0         0       2   
MAASTRICHT      3         0         0       0           0         0       4   
MADRID        129       106         3     145           0         3      37   
MUNCHENB   

##### Softmax is not producing good results, trial with tanh, sigmoid, and relu

In [31]:
epochs = 30
batch_size = 16
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [32]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 14, 128)             │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 14, 16)              │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 7, 16)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 15)                  │           1,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,191 (24.18 KB)

 Trainable params: 6,191 (24.18 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1989 - loss: 23.9055
Epoch 2/30
1076/1076 - 2s - 2ms/step - accuracy: 0.2939 - loss: 24.3467
Epoch 3/30
1076/1076 - 3s - 3ms/step - accuracy: 0.2998 - loss: 24.3439
Epoch 4/30
1076/1076 - 3s - 2ms/step - accuracy: 0.2981 - loss: 24.3447
Epoch 5/30
1076/1076 - 3s - 3ms/step - accuracy: 0.2756 - loss: 24.3448
Epoch 6/30
1076/1076 - 3s - 3ms/step - accuracy: 0.2744 - loss: 24.3447
Epoch 7/30
1076/1076 - 3s - 2ms/step - accuracy: 0.2716 - loss: 24.3447
Epoch 8/30
1076/1076 - 3s - 2ms/step - accuracy: 0.2699 - loss: 24.3447
Epoch 9/30
1076/1076 - 3s - 3ms/step - accuracy: 0.2658 - loss: 24.3448
Epoch 10/30
1076/1076 - 3s - 3ms/step - accuracy: 0.2625 - loss: 24.3448
Epoch 11/30
1076/1076 - 3s - 2ms/step - accuracy: 0.2531 - loss: 24.3438
Epoch 12/30
1076/1076 - 3s - 2ms/step - accuracy: 0.2395 - loss: 24.3457
Epoch 13/30
1076/1076 - 3s - 3ms/step - accuracy: 0.2200 - loss: 24.3447
Epoch 14/30
1076/1076 - 3s - 2ms/step - accuracy: 0.2082 - l

In [35]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [36]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BASEL  BELGRADE
True                       
BASEL        3681         1
BELGRADE     1092         0
BUDAPEST      214         0
DEBILT         82         0
DUSSELDORF     29         0
HEATHROW       82         0
KASSEL         11         0
LJUBLJANA      61         0
MAASTRICHT      9         0
MADRID        458         0
MUNCHENB        8         0
OSLO            5         0
STOCKHOLM       4         0
VALENTIA        1         0


##### Better loss but accuracy is low, lets try adjusting layers

In [37]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [38]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 14, 64)              │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 14, 16)              │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 7, 16)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 15)                  │           1,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0098 - loss: 24.5516
Epoch 2/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0092 - loss: 25.5940
Epoch 3/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0232 - loss: 25.1277
Epoch 4/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0390 - loss: 21.5438
Epoch 5/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0390 - loss: 21.5457
Epoch 6/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0390 - loss: 21.5195
Epoch 7/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0392 - loss: 21.5036
Epoch 8/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0392 - loss: 21.5251
Epoch 9/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0393 - loss: 21.4624
Epoch 10/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0394 - loss: 21.4324
Epoch 11/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0396 - loss: 21.4146
Epoch 12/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0398 - loss: 21.4080
Epoch 13/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0399 - loss: 21.4118
Epoch 14/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0402 - l

In [41]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [42]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BUDAPEST  DUSSELDORF  HEATHROW  KASSEL  LJUBLJANA  MADRID  \
True                                                                    
BASEL           1952          37        62     441        155     152   
BELGRADE        1078           0        10       0          0       0   
BUDAPEST         214           0         0       0          0       0   
DEBILT            82           0         0       0          0       0   
DUSSELDORF        29           0         0       0          0       0   
HEATHROW          79           0         0       0          0       1   
KASSEL            11           0         0       0          0       0   
LJUBLJANA         56           2         0       1          0       1   
MAASTRICHT         6           1         1       0          0       1   
MADRID           289           1         2      19          1      50   
MUNCHENB           8           0         0       0          0       0   
OSLO      

##### Loss is not is stable but not as low as I would like, however accuracy is much higher. Trial with sigmoid and relu

In [43]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [44]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 14, 64)              │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 14, 16)              │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 7, 16)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 15)                  │           1,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1308 - loss: 11868.9551
Epoch 2/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1268 - loss: 121610.9453
Epoch 3/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1214 - loss: 421748.9062
Epoch 4/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1199 - loss: 928238.4375
Epoch 5/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1187 - loss: 1630950.3750
Epoch 6/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1192 - loss: 2556885.7500
Epoch 7/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1251 - loss: 3708399.5000
Epoch 8/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1210 - loss: 5139717.5000
Epoch 9/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1251 - loss: 6844117.5000
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1275 - loss: 8778984.0000
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1271 - loss: 11178842.0000
Epoch 12/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1296 - loss: 13838953.0000
Epoch 13/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1275 - loss: 16886574.00

In [47]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [48]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  VALENTIA
True                       
BASEL        3679         3
BELGRADE     1092         0
BUDAPEST      214         0
DEBILT         82         0
DUSSELDORF     29         0
HEATHROW       82         0
KASSEL         11         0
LJUBLJANA      61         0
MAASTRICHT      9         0
MADRID        458         0
MUNCHENB        8         0
OSLO            5         0
STOCKHOLM       4         0
VALENTIA        1         0
